In [ ]:
!pip install antigranular


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Harvard OpenDP Hackathon")

Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Connected to Antigranular server session id: 05dc0bf9-74c8-40b1-a07d-e2bb6c624d80, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
%%ag
health = health_organisation_dataset
flight = flight_company_dataset

In [ ]:
%%ag
# Printing the column names using ag_print
ag_print(f"Flight Dataset \n {flight.columns} \n")
ag_print(f"Health Datset \n {health.columns}")

Flight Dataset 
 ['flight_number', 'flight_date', 'flight_from', 'flight_to', 'passenger_firstname', 'passenger_lastname', 'passenger_date_of_birth', 'passenger_phone_number', 'passenger_email_address'] 

Health Datset 
 ['patient_firstname', 'patient_lastname', 'patient_date_of_birth', 'patient_phone_number', 'patient_email_address', 'covidtest_date', 'covidtest_result', 'patient_address']



In [ ]:
%%ag
# Lets remove those passenger records who tested negative.
health['covidtest_result'] = health['covidtest_result'].where(health['covidtest_result'] == 'positive')
health = health.dropna()


In [ ]:
# %%ag
# from op_pandas import PrivateDataFrame , PrivateSeries

In [ ]:
%%ag
ag_print(f"Total covid positive health records : {health['patient_firstname'].count(eps=0.1)}")

In [ ]:
%%ag
import op_recordlinkage as rl
# A full indexing is a complete cartesian product
indexer  = rl.Index()
indexer.full()

In [ ]:
%%ag
import op_recordlinkage as rl
indexer = rl.Index()
indexer.block('passenger_phone_number','patient_phone_number')
# indexer.block('passenger_lastname','patient_lastname')
candidate_links = indexer.index(flight,health)

In [ ]:
%%ag
# total number of links based on this indexing choice.
ag_print(candidate_links.count(eps=0.1))


In [ ]:
%%ag
compare = rl.Compare()


def cmp1( name1 : str , name2 : str ) -> float:
  if not isinstance(name1,str):
    return 0.0
  if not isinstance(name2,str):
    return 0.0
  # Convert both names to lowercase for a case-insensitive comparison
  name1 = name1.lower()
  name2 = name2.lower()

  # Exact match
  if name1 == name2:
    return 1.0

  # Initial check: E.g. "A. Smith" and "Adam Smith"
  if name1[0] == name2[0] and (name1[1] == '.' or name2[1] == '.'):
    return 0.9

  # Substring check: E.g. "Rob" and "Robert"
  if name1 in name2 or name2 in name1:
    return 0.8

  # Length difference: E.g. "Ann" and "Anne"
  if abs(len(name1) - len(name2)) == 1:
    return 0.7

  return 0.0

# compare.string("passenger_firstname" , "patient_firstname" ,method='cosine', label="firstname")
compare.custom(cmp1,"passenger_firstname","patient_firstname",label="firstname")
compare.string("passenger_lastname","patient_lastname" ,method='jaro', label="lastname")
# compare.exact("passenger_phone_number","patient_phone_number", label="phno")


# Using a custom compare rule.
# custom functions are executed in isolated environment.
import datetime
def cmp(date_str1:str , date_str2:str)->int: # datetime and regex are pre-imported in isolated environment.
    # Convert date strings to datetime objects
    date1 = datetime.datetime.strptime(date_str1, "%Y-%m-%d")
    date2 = datetime.datetime.strptime(date_str2, "%Y-%m-%d")

    # Calculate the absolute difference in days
    days_apart = (date2 - date1).days
    # Check if the dates are within two weeks (14 days) apart
    if days_apart <= 14:
        return 2
    else:
        return 0

compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")

In [ ]:
%%ag
features = compare.compute(candidate_links,flight,health)

In [ ]:
%%ag
linked_df = compare.get_match(4)

In [ ]:
%%ag
res = linked_df[["l_flight_number"]]
x = submit_predictions(res)

score: {'leaderboard': 0.4965, 'logs': {'LIN_EPS': -0.0034999999999999996, 'MCC': 0.5}}



In [ ]:
session.privacy_odometer()

{'total_epsilon_used': 0.7,
 'total_delta_used': 0.0,
 'library_costs': {'op_pandas': {'total_delta': 0,
   'total_epsilon': 0.7,
   'total_requests': 52}},
 'dataset_costs': {'Health Organisation Dataset': {'delta': 0, 'eps': 0.7}}}